# Reglas de asociación para clasificación



## Sobre el paquete arulesCBA:

El paquete **arulesCBA** proporciona funcionalidades para la extracción de reglas de asociación utilizando el algoritmo de Clasificación Basada en Asociaciones (CBA) en R. Se puede encontrar la documentación oficial del paquete [aquí](https://cran.r-project.org/web/packages/arulesCBA/arulesCBA.pdf).

Algunas de las características y funcionalidades principales del paquete incluyen:

- **Extracción de reglas de asociación**: Utiliza el algoritmo CBA para extraer reglas de asociación a partir de conjuntos de datos.

- **Generación de modelos de clasificación**: Construye modelos de clasificación basados en reglas de asociación extraídas.

- **Interpretación de reglas**: Facilita la interpretación de las reglas de asociación generadas para comprender las relaciones entre las variables en los datos.

Es importante tener en cuenta que el éxito del análisis de reglas de asociación con **arulesCBA** depende en gran medida de la calidad del preprocesamiento de datos realizado previamente.

## Preprocesamiento:

Este tipo de análisis requiere un preprocesamiento de datos adecuado para garantizar la calidad de las reglas de asociación extraídas. Algunas de las tareas de preprocesamiento que se pueden realizar incluyen:

- **Limpieza de datos**: Eliminar datos faltantes, duplicados y cualquier otro ruido que pueda afectar la calidad de las reglas de asociación.
- **Transformación de datos**: Convertir los datos en un formato de variables categóricas tanto para variables binarias como numéricas.

#### Librerias requeridas

In [37]:
library(tidyverse)
source("Preprocesamiento/utils.R")
source("Preprocesamiento/plots.R")

#### Lectura del dataset

In [38]:
data <- read.csv("../data/Statlog_weights.csv", header = TRUE, sep = ",")

#### Estructura de los datos

In [39]:
data %>% str()

'data.frame':	1000 obs. of  23 variables:
 $ Class                                                   : int  1 2 1 1 2 1 1 1 1 2 ...
 $ Status.of.existing.checking.account                     : chr  "< 0 DM" "0 - 200 DM" "no checking account" "< 0 DM" ...
 $ Duration.in.month                                       : int  6 48 12 42 24 36 24 36 12 30 ...
 $ Credit.history                                          : chr  "critical account" "existing credits paid" "critical account" "existing credits paid" ...
 $ Purpose                                                 : chr  "radio/television" "radio/television" "education" "furniture/equipment" ...
 $ Credit.amount                                           : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ Savings.account.bonds                                   : chr  "unknown/no savings" "< 100 DM" "< 100 DM" "< 100 DM" ...
 $ Present.employment.since                                : chr  ">= 7 years" "1 - 4 years" "4 - 7 year

#### Información de los datos

In [40]:
data %>% head()

,Class,Status.of.existing.checking.account,Duration.in.month,Credit.history,Purpose,Credit.amount,Savings.account.bonds,Present.employment.since,Installment.rate.in.percentage.of.disposable.income,Other.debtors...guarantors,...,Other.installment.plans,Housing,Number.of.existing.credits.at.this.bank,Job,Number.of.people.being.liable.to.provide.maintenance.for,Telephone,Foreign.worker,Gender,Marital.Status,Weights
,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,...,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1,< 0 DM,6,critical account,radio/television,1169,unknown/no savings,>= 7 years,4,none,...,none,own,2,skilled employee,1,"yes, registered",yes,male,single,1
2,2,0 - 200 DM,48,existing credits paid,radio/television,5951,< 100 DM,1 - 4 years,2,none,...,none,own,1,skilled employee,1,none,yes,female,divorced/separated/married,1
3,1,no checking account,12,critical account,education,2096,< 100 DM,4 - 7 years,2,none,...,none,own,1,unskilled resident,2,none,yes,male,single,1
4,1,< 0 DM,42,existing credits paid,furniture/equipment,7882,< 100 DM,4 - 7 years,2,guarantor,...,none,for free,1,skilled employee,2,none,yes,male,single,1
5,2,< 0 DM,24,delay in paying,car (new),4870,< 100 DM,1 - 4 years,3,none,...,none,for free,2,skilled employee,2,none,yes,male,single,1
6,1,no checking account,36,existing credits paid,education,9055,unknown/no savings,1 - 4 years,2,none,...,none,for free,1,unskilled resident,2,"yes, registered",yes,male,single,1


#### Consulta de valores nulos

In [41]:
col_na_summary(data)

,NA_count
,<dbl>
Class,0
Status.of.existing.checking.account,0
Duration.in.month,0
Credit.history,0
Purpose,0
Credit.amount,0
Savings.account.bonds,0
Present.employment.since,0
Installment.rate.in.percentage.of.disposable.income,0


#### Convertir a valores categóricos los numéricos

In [42]:
data <- cut_numeric_columns(data, 5)

#### Modificación del formato

In [43]:
data <- format_adapter(data)

In [44]:
data %>% head()

Class,Status.of.existing.checking.account,Duration.in.month,Credit.history,Purpose,Credit.amount,Savings.account.bonds,Present.employment.since,Installment.rate.in.percentage.of.disposable.income,Other.debtors...guarantors,...,Other.installment.plans,Housing,Number.of.existing.credits.at.this.bank,Job,Number.of.people.being.liable.to.provide.maintenance.for,Telephone,Foreign.worker,Gender,Marital.Status,Weights
1,< 0 DM,(3~12],critical account,radio/television,(249~1.26e+03],unknown/no savings,>= 7 years,(3~4],none,...,none,own,(0~2],skilled employee,(0~2],yes~ registered,yes,male,single,1.0000000
2,0 - 200 DM,(30~72],existing credits paid,radio/television,(4.72e+03~1.84e+04],< 100 DM,1 - 4 years,(0~2],none,...,none,own,(0~2],skilled employee,(0~2],none,yes,female,divorced/separated/married,1.0000000
1,no checking account,(3~12],critical account,education,(1.91e+03~2.85e+03],< 100 DM,4 - 7 years,(0~2],none,...,none,own,(0~2],unskilled resident,(0~2],none,yes,male,single,1.0000000
1,< 0 DM,(30~72],existing credits paid,furniture/equipment,(4.72e+03~1.84e+04],< 100 DM,4 - 7 years,(0~2],guarantor,...,none,for free,(0~2],skilled employee,(0~2],none,yes,male,single,1.0000000
2,< 0 DM,(15~24],delay in paying,car (new),(4.72e+03~1.84e+04],< 100 DM,1 - 4 years,(2~3],none,...,none,for free,(0~2],skilled employee,(0~2],none,yes,male,single,1.0000000
1,no checking account,(30~72],existing credits paid,education,(4.72e+03~1.84e+04],unknown/no savings,1 - 4 years,(0~2],none,...,none,for free,(0~2],unskilled resident,(0~2],yes~ registered,yes,male,single,1.0000000


#### Escritura de los datos en formato rCBA

In [45]:
write.csv(data, "../data/Statlog_rCBA.csv", row.names = FALSE)